In [1]:
import jupyter_dashboards

In [2]:
import ipywidgets as widgets

In [3]:
import os

In [4]:
from IPython.display import display
import pandas as pd
import numpy as np

In [29]:
! pip install jupyter_dashboards
! jupyter dashboards quick-setup --sys-prefix

/bin/sh: 1: pip: not found
Installing /home/eiroamd/.local/lib/python3.6/site-packages/jupyter_dashboards/nbextension -> jupyter_dashboards
Traceback (most recent call last):
  File "/home/eiroamd/.local/bin/jupyter-dashboards", line 11, in <module>
    sys.exit(main())
  File "/home/eiroamd/.local/lib/python3.6/site-packages/jupyter_dashboards/extensionapp.py", line 203, in main
    ExtensionApp.launch_instance()
  File "/home/eiroamd/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/eiroamd/.local/lib/python3.6/site-packages/jupyter_dashboards/extensionapp.py", line 199, in start
    super(ExtensionApp, self).start()
  File "/home/eiroamd/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 305, in start
    return self.subapp.start()
  File "/home/eiroamd/.local/lib/python3.6/site-packages/jupyter_dashboards/extensionapp.py", line 122, in start
    install.start()
  File "/home/eiroamd

In [5]:
! jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/eiroamd/Documents/IH/Ironhack Data Analytics.json"

In [7]:
from google.cloud import bigquery

client = bigquery.Client()

In [8]:
widgets.IntSlider(
    min = 0,
    max = 10,
    step = 1,
    description='Slider',
    value = 3
)

IntSlider(value=3, description='Slider', max=10)

Crear botón:

In [26]:
btn = widgets.Button(description = 'Medium')

In [27]:
display(btn)

Button(description='Medium', style=ButtonStyle())

Para darle función al botón:

In [25]:
def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))

btn.on_click(btn_eventhandler)

Hacer query:

In [12]:
query1 = """
SELECT
  end_station_name,
  usertype,
  gender,
  COUNT(*) as total_trips,
  AVG(tripduration) as avg_duration
FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  end_station_name,
  usertype,
  gender
HAVING
  total_trips > 20000
"""

In [13]:
query_job = client.query(query=query1)
df = query_job.to_dataframe()
df.head(20)

,end_station_name,usertype,gender,total_trips,avg_duration
0,Pershing Square North,Subscriber,male,346683,727.332716
1,Washington St & Gansevoort St,Subscriber,male,107859,812.164140
2,Grand Army Plaza & Central Park S,Subscriber,male,120748,864.819856
3,Christopher St & Greenwich St,Subscriber,male,192827,769.657392
4,Cooper Square & E 7 St,Subscriber,male,51126,645.257912
5,Broadway & E 14 St,Subscriber,male,232964,639.402680
6,W 17 St & 8 Ave,Subscriber,male,145531,649.038494
7,W 26 St & 8 Ave,Subscriber,female,46683,791.748709
8,Hicks St & Montague St,Subscriber,male,39746,783.756906
9,Henry St & Grand St,Subscriber,female,33229,799.331819


Crear función que nos pille los valores únicos:

In [14]:
all = 'ALL'
def unique_values_all(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, all)
    return unique

In [15]:
unique_values_all(df.gender)

['ALL', '', 'female', 'male', 'unknown']

Crear dropdown usando la función como argumento

In [16]:
dropdown_usertype = widgets.Dropdown(options = unique_values_all(df.usertype))
dropdown_gender = widgets.Dropdown(options = unique_values_all(df.gender))

In [17]:
display(dropdown_usertype)

Dropdown(options=('ALL', '', 'Customer', 'Subscriber'), value='ALL')

Función que me filtre los datos dependiendo del valor que haya seleccionado en el dropdown

In [18]:
my_output = widgets.Output()

In [19]:
def df_filtering(gender, usertype):
    my_output.clear_output()
    if (usertype == all) & (gender == all):
        df_filtered = df
    elif (gender == all):
        df_filtered = df[df.usertype == usertype]
    elif (usertype == all):
        df_filtered = df[df.gender == gender]
    else:
        df_filtered = df[(df.usertype == usertype) & (df.gender == gender)]
    
    with my_output:
        display(df_filtered)

def dropdown_gender_eventhandler(change):
    df_filtering(change.new, dropdown_usertype.value)

def dropdown_usertype_eventhandler(change):
    df_filtering(dropdown_gender.value, change.new)

# usamos la función observe, que me pilla los cambios en el botón
dropdown_gender.observe(dropdown_gender_eventhandler, names='value')
dropdown_usertype.observe(dropdown_usertype_eventhandler, names='value')

display(dropdown_gender)
display(dropdown_usertype)

Dropdown(options=('ALL', '', 'female', 'male', 'unknown'), value='ALL')

Dropdown(options=('ALL', '', 'Customer', 'Subscriber'), value='ALL')

In [20]:
display(my_output)

Output()

In [21]:
df_filtering('male',all)